In [ ]:
# imports

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests


def get_bike_stations(city):
    # Step 1: Get the networks data
    networks_url = "http://api.citybik.es/v2/networks"
    response = requests.get(networks_url)
    data = response.json()
    
    # Step 2: Find the network ID for the given city
    network_id = None
    for network in data['networks']:
        if network['location']['city'] == city:
            latitude = network["location"]["latitude"]
            longitude = network["location"]["longitude"]
            network_id = network['id']
            break
    
    # Step 3: Retrieve bike station data for the network
    if network_id:
        network_url = f"http://api.citybik.es/v2/networks/{network_id}"
        response = requests.get(network_url)
        data = response.json()
        
        # Step 4: Extract the bike stations
        bike_stations = data['network']['stations']
        return bike_stations,latitude,longitude
    
    return None,None,None

# Specify the city you want to retrieve bike stations for
city = "Boise, ID"

# Call the function to get bike stations and their longitude and latitude in the specified city
stations,latitude,longitude = get_bike_stations(city)

fs_url = f"https://api.foursquare.com/v3/places/nearby?fields=name%2Cgeocodes%2Crating&ll={latitude}%2C{longitude}&query=bar%2Crestaurant"

headers = {
    "accept": "application/json",
    "Authorization": "FOURSQUARE_API" # api key
}

fs_response = requests.get(fs_url, headers=headers)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
import pandas as pd

# Create a Pandas dataframe from the bike stations data
if stations:
    cityb_df = pd.DataFrame(stations)
    print(f"DataFrame of available bike stations in {city}:\n")
    cityb_df['dummy'] = 1
    print(cityb_df)
else:
    print(f"No bike stations found in {city}.")
    exit

fs_data = fs_response.json()

#parsing through the data from foursquare

print(fs_data)

Put your parsed results into a DataFrame

In [ ]:
foursquare_df = pd.DataFrame(fs_data)

foursquare_normalized = pd.json_normalize(foursquare_df['results'])

foursquare_df = pd.concat([foursquare_df.drop('results', axis=1), foursquare_normalized], axis=1)
fieldstodrop = ['geocodes.drop_off.longitude','geocodes.drop_off.latitude','geocodes.roof.longitude','geocodes.roof.latitude',]
foursquare_df = foursquare_df.drop(fieldstodrop,axis=1)
#dummy selected as a common key attribute for joining data
foursquare_df['dummy'] = 1

#checking the dataframe
print(foursquare_df)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
y_url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&term=restaurants%2Cbars&radius=1000&sort_by=best_match&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer YELP_API"
}

y_response = requests.get(y_url, headers=headers)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
y_data = y_response.json()
businesses = y_data['businesses']

for business in businesses:
    name = business['name']
    rating = business['rating']
    coordinates = business['coordinates']
    print(name,rating,coordinates)

Put your parsed results into a DataFrame

import requests
import pandas as pd

url = "https://api.yelp.com/v3/businesses/search?latitude=43.62283460935435&longitude=-116.23483657836914&radius=1000&sort_by=best_match&limit=50"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer YELP_API"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Request was successful, process the response data
    data = response.json()
    businesses = data.get("businesses", [])  # Extract the "businesses" data from the response
    
    # Initialize lists to store the desired details
    ratings = []
    names = []
    locations = []
    
    # Extract the desired details from each business, up to the top 50
    counter = 0
    for business in businesses:
        if counter >= 50:
            break
        
        ratings.append(business.get("rating", None))
        names.append(business.get("name", None))
        locations.append(business.get("location", {}).get("address1", None))
        
        counter += 1
    # Create a DataFrame with the extracted details
    yelp_data = pd.DataFrame({
        "Name": names,
        "Rating": ratings,
        "Location": locations
    })
    
    # Optionally, you can further process or analyze the DataFrame as needed
    print(yelp_data)
else:
    # Request encountered an error
    print("Error:", response.status_code)

    

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [ ]:

# In my analysis both yelp and foursquare proved to be very good for extracting data
# Both had similar quality results however foursquare had more flexibility in it's fields and
# provided much more data for every location, hence i chose it for my model

Get the top 10 restaurants according to their rating

In [ ]:
# Displaying data for yelp
sorted_businesses = sorted(businesses, key=lambda b: b['rating'], reverse=True)

for business in sorted_businesses[:10]:
    name = business['name']
    rating = business['rating']
    coordinates = business['coordinates']
    print(name, rating, coordinates)

#Displaying the data for foursquare
results = fs_data['results']

for result in results:
    name = result['name']
    coordinates = result['geocodes']
    print(name,coordinates)


# In my analysis both yelp and foursquare proved to be very good for extracting data
# Both had similar quality results however foursquare had more flexibility in it's fields and
# provided much more data for every location, hence i chose it for my model